In [ ]:
import pandas as pd
import numpy as np
import fluids as fds

import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

In [ ]:
x = torch.tensor([0,1,2,3])

In [ ]:
x[0]+2

In [ ]:
n = 4

def function(x):
    return x[0]+2*x[1]+3*x[2]+4*x[3]
    # return fds.fittings.K_branch_converging_Crane(D_run=x[0], D_branch=x[1], Q_run=x[2], Q_branch=x[3], angle=90)  # If x is a single vector


class VectorDataset(Dataset):
    def __init__(self, size=1000, ranges=[(0.010, 0.100), (0.001, 0.030), (1*1E-6, 1*1E-5), (1*1E-8, 1*1E-5)]):
        self.size = size
        self.ranges = ranges
        n = len(ranges)

        # Initialize an empty tensor for x
        self.x = torch.empty((size, n))

        # Fill each coordinate with random values according to its range
        for i in range(n):
            lower, upper = ranges[i]
            self.x[:, i] = torch.rand(size) * (upper - lower) + lower

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        x = self.x[idx]
        y = function(x).float()
        return x, y
    
dataset = VectorDataset()

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n, 32)  # Input layer with 1 feature, 16 neurons
        self.fc2 = nn.Linear(32, 64) # Hidden layer with 16 neurons
        self.fc3 = nn.Linear(64, 64)  # Output layer with 1 output
        self.fc4 = nn.Linear(64, 1)  # Output layer with 1 output

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = Net()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 200

for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        inputs, targets = inputs.float(), targets.float()
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.squeeze()

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()


    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

In [ ]:
num_epochs = 2000

for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        inputs, targets = inputs.float(), targets.float()
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.squeeze()

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()


    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

In [ ]:
model.eval()
test_loss = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        outputs = outputs.squeeze()
        loss = criterion(outputs, targets)
        test_loss += loss.item()
test_loss /= len(test_loader)
print(f'Test Loss: {test_loss}')

In [ ]:
# Predict for a new value, e.g., x = 5
x_new = torch.tensor([0.018,0.003,1*1E-7,1*1E-6], dtype=torch.float32)  # Convert to float and add batch dimension
model.eval()
with torch.no_grad():
    prediction = model(x_new)
print('Predicted value:', prediction.item())

In [ ]:
function(x_new)